In [1]:
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
import psycopg2
import os

from dotenv import load_dotenv

In [2]:
load_dotenv()

def create_connection():
    
    DATABASE = os.getenv("DATABASE")
    HOST = os.getenv("HOST")  
    USER = os.getenv("USER")
    PASSWORD = os.getenv("PASSWORD")
    PORT = os.getenv("PORT")
    
    connection = psycopg2.connect(
        host=HOST, 
        database=DATABASE, 
        user=USER, 
        password=PASSWORD,
        port=PORT
    )
    
    return connection

## Extraction et mise en forme des données 

In [3]:
df = pd.read_csv('../datas/data_sport_10_10_24 - data_sport.csv', delimiter=',',on_bad_lines='skip', decimal='.')
df.head()

,Unnamed: 0,Date,Sport,Séance,Exercice,Durée,poid,Nbr_de_repetitions,Commentaire,id
0,0,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,0
1,1,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,1
2,2,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,2
3,3,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,10.0,NaN,3
4,4,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,9.0,NaN,4


In [4]:
def clean_data(x): 
    x = str(x)
    x = x.replace(',','.')
    x = x.replace('\xa0', '')
    return float(x)



In [5]:
df["Sport"].nunique()

1

In [6]:

df["poid"] = df["poid"].apply(clean_data) 
df["poid"] = df["poid"].astype("float")
df["Date"] =  pd.to_datetime(df["Date"])



In [7]:
df["Nbr_de_repetitions"] = df["Nbr_de_repetitions"].replace('max',5 )
df["Nbr_de_repetitions"] = df["Nbr_de_repetitions"].apply(clean_data) 
df["Nbr_de_repetitions"] = df["Nbr_de_repetitions"].astype("float")

In [8]:
# correction of ValueError: could not convert string to float: '1h30'
df["Durée"] =  df["Durée"].replace('1h30', 90)

In [9]:
df["Durée (min)"] = df["Durée"].replace("Nan", np.nan)
df["Durée (min)"] = df["Durée"].replace("7,2", 7.2)
df["Durée"] = df["Durée"].astype("float")


In [10]:
df = df[["Date","Sport","Séance","Exercice","Durée","poid","Nbr_de_repetitions",'Commentaire']]

In [11]:
df["id"] = df.index

## Je m'assure que tous les Musculations_exerices soit bien conforme à ce que je veux en base de donnée

In [12]:
df_musculation_exercices = df.loc[df["Sport"]=="Musculation"]
df_musculation_exercices.head()

,Date,Sport,Séance,Exercice,Durée,poid,Nbr_de_repetitions,Commentaire,id
0,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,0
1,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,1
2,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,2
3,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,10.0,NaN,3
4,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,9.0,NaN,4


In [13]:
unique_musculation_exerice = list(df_musculation_exercices["Exercice"].unique())
unique_musculation_exerice = sorted(unique_musculation_exerice)
for x in unique_musculation_exerice:
    print(x)

Abdominal
Abdos Inclinés
Abdos Machine
Abs Wheel
Arm Curl
Back Extension
Biceps Curl Allongé
Biceps Curl Assis
Biceps Curl Poulie
Biceps Curl Prise Marteau
Biceps Curl Prise Marteau à la Poulie
Biceps Curl à la barre
Biceps Curls
Chest Press
Converging Chest Press
Converging Shoulder Press
Dip Assist
Dips
Diverging Lat Pull Down
Diverging Seated Row
Diverging Seated Row Prise Verticale
Développer Militaire à la Smith Machine
Développé Couché
Développé Couché Haltères
Développé Couché Incliné Bas
Développé Couché Incliné Haut
Développé Couché Prise Serrée
Développé Couché Rapproché
Développé Incliné
Développé Incliné Haltères
Développé Militaire
Développé Militaire Haltères
Développé Militaire à la Smith Machine
Développé Militaire à la barre
Développé militaires Haltères
Face Pull Down à la poulie
Fentes
Hip Abduction
Hip Thrust
Lat Pull
Lat Pull Prise Poignée
Lateral Raise Poulie
Leg Extension
Leg Press
Low Row
Lower Back Bench
Machine Abdominaux
Mollet sur Leg Press
Pec Fly
Pectoraux

In [14]:
replace_dict_1 = {
    "Abdos Inclinés": "Abdos Machine",
    "Biceps Curl Haltères Assis": "Biceps Curl Assis",
    "Biceps Curl Prise marteau": "Biceps Curl Prise Marteau",
    "Biceps Curl à la Poulie": "Biceps Curl Poulie",
    "Biceps Curl Assis ": "Biceps Curl Assis",
    "Biceps Curls (allongé puis prise marteau)": "Biceps Curl Allongé",
    "Diverging Seated Row Prise Verticale ": "Diverging Seated Row Prise Verticale",
    "Lower back Bench": "Lower Back Bench",
    "Prône leg Curl": "Prone leg Curl",
    "Élévations Latérales à la poulie": "Élévations Latérales à la Poulie",
    "Élévations Frontales": "Élévations Frontales Haltères",
    "Élévations frontales": "Élévations Frontales Haltères",
    "Élévations latérales": "Élévations Latérales Haltères",
    "Élévation Latérale.haltères": "Élévations Latérales Haltères",
    "Diverging Seated Row prise verticale": "Diverging Seated Row Prise Vertical",
    "Biceps curl assis": "Biceps curl Assis",
    "Élévations Latérales": "Élévations Latérales Haltères",
    "low row": "Low Row",
    "mollet sur leg press": "Mollets sur Leg Press",
    "chest press": "Chest Press",
    "élévation frontale disque": "Élévation Frontale Disque",
    "Élévation frontale poulie": "Élévation Frontale Poulie",
    "Tricpes Poulie Inversée": "Triceps à la Poulie Inversée",
}

replace_dict_2 = {
    "Abdos Machine": "Abdos Machine",
    "Abdos inclinés": "Abdos Machine",
    "Abs Wheel": "Abs Wheel",
    "Arm Curl": "Arm Curl",
    "Back Extension": "Back Extension",
    "Biceps Curl Allongé": "Biceps Curl Allongé",
    "Biceps Curls (allongé puis prise marteau)": "Biceps Curl Allongé",
    "biceps curl allongé": "Biceps Curl Allongé",
    "Biceps Curl Assis": "Biceps Curl Assis",
    "Biceps Curl Haltères Assis": "Biceps Curl Assis",
    "Biceps Curl Assis ": "Biceps Curl Assis",
    "biceps curl assis": "Biceps Curl Assis",
    "Biceps Curl Poulie": "Biceps Curl Poulie",
    "Biceps Curl à la Poulie": "Biceps Curl Poulie",
    "biceps curl poulie": "Biceps Curl Poulie",
    "Biceps Curl Prise Marteau": "Biceps Curl Prise Marteau",
    "Biceps Curl Prise marteau": "Biceps Curl Prise Marteau",
    "Biceps Curl Prise Marteau à la Poulie": "Biceps Curl Prise Marteau à la Poulie",
    "Biceps Curls": "Biceps Curls",
    "Biceps Curls à la Poulie": "Biceps Curl Poulie",
    "Biceps curl Assis": "Biceps Curl Assis",
    "Biceps curl assis prise marteau": "Biceps Curl Prise Marteau",
    "Biceps curl à la poulie": "Biceps Curl Poulie",
    "Biceps curl à la barre": "Biceps Curl à la barre",
    "Chest Press": "Chest Press",
    "chest press": "Chest Press",
    "Converging Chest Press": "Converging Chest Press",
    "Converging Shoulder Press": "Converging Shoulder Press",
    "converging shoulder press": "Converging Shoulder Press",
    "Developpé Militaire": "Développé Militaire",
    "Développer Militaire à la Barre": "Développé Militaire à la barre",
    "Développé Militaire Haltères": "Développé Militaire Haltères",
    "Développé Militaire haltères": "Développé Militaire Haltères",
    "Développé Militaire à la barre": "Développé Militaire à la barre",
    "Développé militaire haltères": "Développé Militaire Haltères",
    "Développé militaire à la Smith Machine": "Développé Militaire à la Smith Machine",
    "Developpé Militaire": "Développé Militaire",
    "Dip Assist": "Dip Assist",
    "Dips": "Dips",
    "Diverging Lat Pull Down": "Diverging Lat Pull Down",
    "Diverging Seated Row": "Diverging Seated Row",
    "Diverging Seated Row Prise Vertical": "Diverging Seated Row Prise Verticale",
    "Diverging Seated Row Prise Verticale ": "Diverging Seated Row Prise Verticale",
    "Diverging Seated Row prise verticale": "Diverging Seated Row Prise Verticale",
    "Développer Incliné Haltères": "Développé Incliné Haltères",
    "Développé Couché": "Développé Couché",
    "Développé Couché ": "Développé Couché",
    "Développé Couché Prise Serrée": "Développé Couché Prise Serrée",
    "Développé Couché haltère": "Développé Couché haltère",
  "Développé Couché incliné bas": "Développé Couché incliné bas",
    "Développé Couché incliné haut": "Développé Couché incliné haut",
    "Développé couché incliné haut": "Développé Couché incliné haut",
    "Développé couché prise serrée ": "Développé Couché Prise Serrée",
    "Développé incliné": "Développé Incliné",
    "Ecartés Poulie": "Ecartés Poulie",
    "Ecartés Poulie Bas": "Ecartés Poulie",
    "Face Pull Down à la poulie": "Face Pull Down à la poulie",
    "Fentes": "Fentes",
    "Hip Abduction": "Hip Abduction",
    "Hip Abduction vers exterieur": "Hip Abduction",
    "Hip Abduction vers intérieur": "Hip Abduction",
    "Hip Adduction": "Hip Adduction",
    "Hip Thrust": "Hip Thrust",
    "Hip Trust": "Hip Thrust",
    "Hip trust": "Hip Thrust",
    "Lat Pull": "Lat Pull",
    "Lat Pull ": "Lat Pull",
    "Lat Pull prise Poignée": "Lat Pull prise Poignée",
    "Lat Pull prise poignée": "Lat Pull prise Poignée",
    "Lateral Raise Poulie": "Lateral Raise Poulie",
    "Leg Extension": "Leg Extension",
    "leg extension": "Leg Extension",
    "Leg Press": "Leg Press",
    "Legs Extension": "Leg Extension",
    "Levées laterale poulie": "Lateral Raise Poulie",
    "Levés Frontales Poulie": "Élévation Frontale Poulie",
    "Low Row": "Low Row",
    "low row": "Low Row",
    "Lower Back Bench": "Lower Back Bench",
    "Lower back bench": "Lower Back Bench",
    "Machine Abdominaux": "Machine Abdominaux",
    "Mollets sur Leg Press": "Mollets sur Leg Press",
    "mollet sur leg press": "Mollets sur Leg Press",
    "Pec Fly": "Pec Fly",
    "pec fly": "Pec Fly",
    "Pectoraux coulis vers le bas": "Pectoraux à la poulie bas",
    "Pectoraux à la poulie": "Pectoraux à la poulie",
    "pectoraux à la coulis": "Pectoraux à la poulie",
    "Pectoraux à la poulie bas": "Pectoraux à la poulie bas",
    "Pectoraux à la poulie haut": "Pectoraux à la poulie haut",
    "Prone leg Curl": "Prone leg Curl",
    "Prône leg curl": "Prone leg Curl",
    "Rear Delt": "Rear Delt",
    "Rotary Torso": "Rotary Torso",
    "Rowing Bar": "Rowing Bar",
    "Rowing Menton": "Rowing Menton",
    "Rowing buste penché": "Rowing buste penché",
    "Rowing debout": "Rowing debout",
    "rowing debout": "Rowing debout",
    "Seated Row": "Seated Row",
    "Seated Row Prise Verticale": "Seated Row Prise Verticale",
    "Shoulder press": "Shoulder press",
    "Squat": "Squat",
    "Tirage Menton": "Tirage Menton",
    "Tirage Verticale": "Tirage Verticale",
    "Traction avec Élastique": "Traction avec Élastique",
    "traction avec élastique": "Traction avec Élastique",
    "Triceps Double Chassées Croisé à la poulie": "Triceps Double Chassées Croisé à la poulie",
    "Triceps Doubles Chassées à la Poulie": "Triceps Doubles Chassées à la Poulie",
    "triceps doubles chassées poulie": "Triceps Doubles Chassées à la Poulie",
    "Triceps Pull Downs Poulie": "Triceps Pull Downs Poulie",
    "Triceps poulie ": "Triceps à la Poulie",
    "Triceps poulie inversées": "Triceps à la Poulie Inversée",
    "Triceps à la Poulie": "Triceps à la Poulie",
    "Triceps à la Poulie Inversée": "Triceps à la Poulie Inversée",
    "Tricpes Double Chassés Poulie": "Triceps Doubles Chassées à la Poulie",
    "Élévation Frontale Disque": "Élévation Frontale Disque",
    "élévation frontale disque": "Élévation Frontale Disque",
    "Élévation Frontale Poulie": "Élévation Frontale Poulie",
    "élévation frontale poulie": "Élévation Frontale Poulie",
    "Élévations Frontales Haltères": "Élévations Frontales Haltères",
    "Élévations Frontales": "Élévations Frontales Haltères",
    "Élévations frontales": "Élévations Frontales Haltères",
    "Élévations Latérales Haltères": "Élévations Latérales Haltères"}
df_musculation_exercices["Exercice"] = df_musculation_exercices["Exercice"].replace(replace_dict_2)


In [15]:
replace_dict_3 = {   "Abdos Machine": "Abdos Machine",
    "Abdos inclinés": "Abdos Machine",
    "Abs Wheel": "Abs Wheel",
    "Arm Curl": "Arm Curl",
    "Back Extension": "Back Extension",
    "Banc à Lombaire": "Back Extension",
    "Biceps Curl Allongé": "Biceps Curl Allongé",
    "biceps curl allongé": "Biceps Curl Allongé",
    "Biceps Curl Assis": "Biceps Curl Assis",
    "Biceps Curl Poulie": "Biceps Curl Poulie",
    "Biceps Curl Prise Marteau": "Biceps Curl Prise Marteau",
    "Biceps Curl Prise Marteau à la Poulie": "Biceps Curl Prise Marteau à la Poulie",
    "Biceps Curls": "Biceps Curls",
    "Biceps curl Assis": "Biceps Curl Assis",
    "Biceps curl à la poulie": "Biceps Curl Poulie",
    "Biceps curl à la barre": "Biceps Curl à la barre",
    "Chest Press": "Chest Press",
    "Converging Chest Press": "Converging Chest Press",
    "Converging Shoulder Press": "Converging Shoulder Press",
    "Dip Assist": "Dip Assist",
    "Dips": "Dips",
    "Diverging Lat Pull Down": "Diverging Lat Pull Down",
    "Diverging Seated Row": "Diverging Seated Row",
    "Diverging Seated Row Prise Vertical": "Diverging Seated Row Prise Verticale",
    "Développé Couché": "Développé Couché",
    "Développé Couché Prise Serrée": "Développé Couché Prise Serrée",
    "Développé Couché haltère": "Développé Couché haltère",
    "Développé Couché incliné bas": "Développé Couché incliné bas",
    "Développé Couché incliné haut": "Développé Couché incliné haut",
    "Développé Incliné": "Développé Incliné",
    "Développé Incliné Haltères": "Développé Incliné Haltères",
    "Développé Militaire": "Développé Militaire",
    "Développé Militaire Haltères": "Développé Militaire Haltères",
    "Développé Militaire à la Smith Machine": "Développé Militaire à la Smith Machine",
    "Développé Militaire à la barre": "Développé Militaire à la barre",
    "Ecartés Poulie": "Ecartés Poulie",
    "Face Pull Down à la poulie": "Face Pull Down à la poulie",
    "Fentes": "Fentes",
    "Hip Abduction": "Hip Abduction",
    "Hip Adduction": "Hip Adduction",
    "Hip Thrust": "Hip Thrust",
    "Lat Pull": "Lat Pull",
    "Lat Pull prise Poignée": "Lat Pull prise Poignée",
    "Lateral Raise Poulie": "Lateral Raise Poulie",
    "Leg Extension": "Leg Extension",
    "Leg Press": "Leg Press",
    "Low Row": "Low Row",
    "Lower Back Bench": "Lower Back Bench",
    "Machine Abdominaux": "Machine Abdominaux",
    "Mollets sur Leg Press": "Mollets sur Leg Press",
    "Pec Fly": "Pec Fly",
    "Pectoraux à la poulie": "Pectoraux à la poulie",
    "Pectoraux à la poulie bas": "Pectoraux à la poulie bas",
    "Pectoraux à la poulie haut": "Pectoraux à la poulie haut",
    "Prone leg Curl": "Prone leg Curl",
    "Rear Delt": "Rear Delt",
    "Rotary Torso": "Rotary Torso",
    "Rowing Bar": "Rowing Bar",
    "Rowing Menton": "Rowing Menton",
    "Rowing buste penché": "Rowing buste penché",
    "Rowing debout": "Rowing debout",
    "Seated Row": "Seated Row",
    "Seated Row Prise Verticale": "Seated Row Prise Verticale",
    "Shoulder press": "Shoulder press",
    "Squat": "Squat",
    "Tirage Menton": "Tirage Menton",
    "Tirage Verticale": "Tirage Verticale",
    "Traction avec Élastique": "Traction avec Élastique",
    "Triceps Double Chassées Croisé à la poulie": "Triceps Double Chassées Croisé à la poulie",
    "Triceps Doubles Chassées à la Poulie": "Triceps Doubles Chassées à la Poulie",
    "Triceps Pull Downs Poulie": "Triceps Pull Downs Poulie",
    "Triceps à la Poulie": "Triceps à la Poulie",
    "Triceps à la Poulie Inversée": "Triceps à la Poulie Inversée",
    "abdos inclinés": "Abdos Machine",
    "biceps curl à la barre": "Biceps Curl à la barre",
    "Élévation Frontale Disque": "Élévation Frontale Disque",
    "Élévation Frontale Poulie": "Élévation Frontale Poulie",
    "Élévations Frontales Haltères": "Élévations Frontales Haltères",
    "Élévations Latérales Haltères": "Élévations Latérales Haltères",
    "Élévations Latérales à la Poulie": "Élévations Latérales à la Poulie"
}

df_musculation_exercices["Exercice"] = df_musculation_exercices["Exercice"].replace(replace_dict_3)


In [16]:
df_musculation_exercices.head()


,Date,Sport,Séance,Exercice,Durée,poid,Nbr_de_repetitions,Commentaire,id
0,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,0
1,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,1
2,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,2
3,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,10.0,NaN,3
4,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,9.0,NaN,4


In [17]:
# Je passe sur chacun des groupes musculaires m'assuré que les exerices de sports soient bien écrit 

In [18]:
df_pec = df_musculation_exercices.loc[df_musculation_exercices["Séance"] =="Pectoraux"]
pec = df_pec["Exercice"].unique()
pec = np.sort(pec)
for x in pec:
    print(x)

Back Extension
Chest Press
Converging Shoulder Press
Dip Assist
Développé Couché
Développé Couché Haltères
Développé Couché Incliné Bas
Développé Couché Incliné Haut
Développé Couché Prise Serrée
Développé Couché Rapproché
Développé Incliné
Développé Incliné Haltères
Pec Fly
Pectoraux à la Poulie Bas
Pectoraux à la Poulie Haut
Pectoraux à la Poulie Haut 
Rotary Torso
Seated Dips
Shoulder Press
Tirage Menton
Triceps Double Chassées Croisé à la Poulie
Triceps Pull Downs Poulie
Triceps à la Poulie Inversée
Élévation Frontale Disque


In [19]:
pec_dict = {"Développé Couché haltère":"Développé Couché Haltères",
            "Développé couché Rapproché":"Développé Couché Rapproché",
            "Pectoraux à la poulie":"Pectoraux à la Poulie Haut ",
            "Shoulder press":"Shoulder Press",
            "Tricpes Poulie Inversée":"Triceps à la Poulie Inversée",
            "Shoulder press":"Shoulder Press",
            "Développé Couché incliné bas":"Développé Couché Incliné Bas",  
            "Développé Couché incliné haut":"Développé Couché incliné Haut",
            "Pectoraux à la poulie haut":"Pectoraux à la Poulie Haut ",  
            "Ecartés Poulie":   "Pectoraux à la Poulie Haut"     
            }

df_musculation_exercices["Exercice"] = df_musculation_exercices["Exercice"].replace(pec_dict)


In [20]:
df2 = df_musculation_exercices.copy()
df2 = df2["Exercice"].unique()
df2 = np.sort(df2)
for x in df2:
    print(x)

Abdominal
Abdos Inclinés
Abdos Machine
Abs Wheel
Arm Curl
Back Extension
Biceps Curl Allongé
Biceps Curl Assis
Biceps Curl Poulie
Biceps Curl Prise Marteau
Biceps Curl Prise Marteau à la Poulie
Biceps Curl à la barre
Biceps Curls
Chest Press
Converging Chest Press
Converging Shoulder Press
Dip Assist
Dips
Diverging Lat Pull Down
Diverging Seated Row
Diverging Seated Row Prise Verticale
Développer Militaire à la Smith Machine
Développé Couché
Développé Couché Haltères
Développé Couché Incliné Bas
Développé Couché Incliné Haut
Développé Couché Prise Serrée
Développé Couché Rapproché
Développé Incliné
Développé Incliné Haltères
Développé Militaire
Développé Militaire Haltères
Développé Militaire à la Smith Machine
Développé Militaire à la barre
Développé militaires Haltères
Face Pull Down à la poulie
Fentes
Hip Abduction
Hip Thrust
Lat Pull
Lat Pull Prise Poignée
Lateral Raise Poulie
Leg Extension
Leg Press
Low Row
Lower Back Bench
Machine Abdominaux
Mollet sur Leg Press
Pec Fly
Pectoraux

In [21]:
remplacement_dict = {
    "Biceps curl allongé": "Biceps Curl Allongé",
    "Biceps curl assis": "Biceps Curl Assis",
    "Diverging lat pulldown": "Diverging Lat Pull Down",
    "Développé Couché incliné Haut": "Développé Couché Incliné Haut",
    "Hip Adduction": "Hip Abduction",
    "Lat Pull prise Poignée": "Lat Pull Prise Poignée",
    "Mollet sur Leg Press": "Mollet sur Leg Press",
    "Mollets sur Leg Press": "Mollet sur Leg Press",
    "Pectoraux à la Poulie Haut": "Pectoraux à la Poulie Haut",
    "Pectoraux à la poulie bas": "Pectoraux à la Poulie Bas",
    "Rowing buste penché": "Rowing Buste Penché",
    "Rowing debout": "Rowing Debout",
    "Triceps Doubles Chassées à la Poulie": "Triceps Double Chassées Croisé à la Poulie",
    "Triceps Double Chassées Croisé à la poulie": "Triceps Double Chassées Croisé à la Poulie",
    "Triceps à la Poulie": "Triceps Pull Downs Poulie",
    "Triceps à la Poulies": "Triceps Pull Downs Poulie",
    "Triceps à la poulie": "Triceps Pull Downs Poulie",
    "Élévation Latérale.Poulies": "Élévations Latérales à la poulie",
    "Élévation frontale poulie": "Élévations Latérales à la poulie",
    "Élévations Latérales à la Poulie": "Élévations Latérales à la poulie",
    "Élévations latérales": "Élévation Latérale Haltères",
    "Élévations Latérales": "Élévation Latérale Haltères",
    "Élévation Latérale.haltères": "Élévation Latérale Haltères"
}


In [22]:
df_musculation_exercices["Exercice"] = df_musculation_exercices["Exercice"].replace(remplacement_dict)


In [23]:
df_musculation_exercices

,Date,Sport,Séance,Exercice,Durée,poid,Nbr_de_repetitions,Commentaire,id
0,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,0
1,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,1
2,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,2
3,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,10.0,NaN,3
4,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,9.0,NaN,4
...,...,...,...,...,...,...,...,...,...
1458,2024-10-10,Musculation,Dos,Biceps Curl Allongé,NaN,8.0,7.0,NaN,1458
1459,2024-10-10,Musculation,Dos,Biceps Curl Allongé,NaN,8.0,7.0,NaN,1459
1460,2024-10-10,Musculation,Dos,Rear Delt,NaN,66.0,10.0,NaN,1460
1461,2024-10-10,Musculation,Dos,Rear Delt,NaN,66.0,10.0,NaN,1461


## Remplacement des données du CSV Par les nouvelles valeurs mises en forme 

In [24]:
df_musculation_exercices.to_csv("../datas/data_sport_10_10_24 - data_sport.csv")

In [25]:
df_musculation_exercices.columns

Index(['Date', 'Sport', 'Séance', 'Exercice', 'Durée', 'poid',
       'Nbr_de_repetitions', 'Commentaire', 'id'],
      dtype='object')

In [32]:
unique_musculation_exerice = df_musculation_exercices[["Exercice","Séance"]]
unique_musculation_exerice = unique_musculation_exerice.drop_duplicates()
print(unique_musculation_exerice.shape[0])
unique_musculation_exerice.to_csv("../datas/musculation_exercices.csv") 

142


In [28]:
df_musculation_area = pd.read_csv("../datas/musculation_exercices_with_muscle_area.csv")

In [93]:
df_musculation_area.shape[0]

76

## Mise en BDD Des données musculations rows 

In [26]:
DATABASE = os.getenv("DATABASE")
HOST = os.getenv("HOST")  
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
PORT = os.getenv("PORT")


In [31]:
df_musculation_exercices.head()

,Date,Sport,Séance,Exercice,Durée,poid,Nbr_de_repetitions,Commentaire,id
0,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,0
1,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,1
2,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,2
3,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,10.0,NaN,3
4,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,9.0,NaN,4


In [39]:
df_musculation_exercices.columns = ["date_seance","sport","seance","exercice","durée","poid","nombre_repetition","commentaire","id"]
df_musculation_exercices.drop("id",axis=1)
df_musculation_exercices['date_seance'] = df_musculation_exercices['date_seance'].dt.strftime('%Y-%m-%d')



AttributeError: Can only use .dt accessor with datetimelike values

In [40]:
df_musculation_exercices["user_id"]  = 19


In [41]:
connection_string = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

table_name ="musculation_rows"
df_musculation_exercices.to_sql(table_name, engine, if_exists='replace', index=False)


463

In [35]:
df_musculation_exercices.shape

(1463, 9)

## Remplissage de la table sport_rows

In [37]:
df["Sport"].value_counts()

Sport
Musculation    1463
Name: count, dtype: int64

In [39]:
df_sport = df.loc[df["Sport"]=="Sport"]

In [39]:
df_sport = df_sport.rename({"Date":"date_seance","Sport":"Sport","Durée":"Durée","Commentaire":"Commentaire","id":"id"},axis=1)

In [40]:
df_sport = df_sport.drop(["Séance","poid","Nbr_de_repetitions"], axis=1)

In [98]:
df_sport["user_id"] =19

In [106]:
df_sport["Exercice"] = df_sport["Exercice"].dropna()

In [107]:
df_sport

,date_seance,Sport,Exercice,Durée,Commentaire,id
26,2024-02-24,Cardio,Vélo Elliptique,30.0,NaN,26
27,2024-02-24,Cardio,Cross Trainning,30.0,NaN,27
28,2024-02-24,Cardio,Tennis,60.0,NaN,28
95,2024-02-03,Cardio,Cross Trainning,NaN,20 pompes \n20 ballons mur en équilibres \n12 ...,95
143,2024-04-03,Cardio,Tennis,60.0,NaN,143
159,2024-07-03,Cardio,Vélo Elliptique,25.0,NaN,159
160,2024-07-03,Cardio,NaN,NaN,Pompes\nEpaules\nAbdo allongée jambes levée \n...,160
188,2024-10-03,Cardio,Tennis,60.0,NaN,188
225,2024-03-24,Cardio,Tennis,60.0,NaN,225
226,2024-03-25,Cardio,Vélo Elliptique,30.0,"intensité 13 calorie 381 distances 2,72 km",226


In [110]:
connection_string = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

table_name ="sport_rows"
df_sport.to_sql(table_name, engine, if_exists='replace', index=False)


38

## Remplissage de la table  sport_exercices 

In [111]:
df_sport = df.loc[df["Sport"]!="Musculation"]

## Remplissage de la table musculation_rows


In [113]:
df.head()

,Date,Sport,Séance,Exercice,Durée,poid,Nbr_de_repetitions,Commentaire,id
0,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,0
1,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,1
2,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,2
3,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,10.0,NaN,3
4,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,9.0,NaN,4


In [125]:
df_musculation = df.loc[df["Sport"]== "Musculation"]
df_musculation["Exercice"] =df_musculation_exercices["Exercice"].replace(replace_dict_3)

/var/folders/p5/j7xh751d45s4z0kzc_3gql0h0000gn/T/ipykernel_16015/26993875.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_musculation["Exercice"] =df_musculation_exercices["Exercice"].replace(replace_dict_3)


In [126]:
df_musculation.columns

Index(['Date', 'Sport', 'Séance', 'Exercice', 'Durée', 'poid',
       'Nbr_de_repetitions', 'Commentaire', 'id'],
      dtype='object')

In [127]:
df_musculation = df_musculation.drop(["Sport","Durée","id"], axis=1)

In [129]:
df_musculation = df_musculation.rename({"Date":"date_seance","musclearea":"Séance","Exercice":"exercice","Nbr_de_repetitions":"nombre_repetition","Commentaire":"comments"})

In [130]:
df_musculation["user_id"] =19

In [131]:
connection_string: str = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

table_name ="musculation_rows"
df_musculation.to_sql(table_name, engine, if_exists='replace', index=False)


232